In [ ]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 7.2 MB/s 
     |████████████████████████████████| 144 kB 62.9 MB/s 
     |████████████████████████████████| 181 kB 53.5 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=e42c416503399f24d89c6b7fb745524376cc2df83bd475836631da06b6ea8493
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
import numpy as np
import wandb 
import time
from helper import *
from initializers import *
from loss import *
from optimizers import *
from activators import *
from forward_propagation import *
from backPropagation import *
from prediction import *
from feedForwardNetwork import *

(trainIn, trainOut), (testIn, testOut) = fashion_mnist.load_data()

N_train_full = trainOut.shape[0]
N_train = int(0.9*N_train_full)
N_validation = int(0.1 * trainOut.shape[0])
N_test = testOut.shape[0]


idx  = np.random.choice(trainOut.shape[0], N_train_full, replace=False)
idx2 = np.random.choice(testOut.shape[0], N_test, replace=False)

trainInFull = trainIn[idx, :]
trainOutFull = trainOut[idx]

trainIn = trainInFull[:N_train,:]
trainOut = trainOutFull[:N_train]

validIn = trainInFull[N_train:, :]
validOut = trainOutFull[N_train:]    

testIn = testIn[idx2, :]
testOut = testOut[idx2]


sweep_config = {
  "name": "Bayesian Sweep",
  "method": "bayes",
  "metric":{
  "name": "validationaccuracy",
  "goal": "maximize"
  },
  "parameters": {
        "max_epochs": {
            "values": [5, 10]
        },

        "initializer": {
            "values": ["RANDOM", "XAVIER"]
        },

        "num_layers": {
            "values": [3, 4, 5]
        },
        
        
        "num_hidden_neurons": {
            "values": [32, 64, 128]
        },
        
        "activation": {
            "values": [ 'TANH',  'SIGMOID', 'RELU']
        },
        
        "learning_rate": {
            "values": [0.001, 0.0001]
        },
        
        
        "weight_decay": {
            "values": [0, 0.0005,0.5]
        },
        
        "optimizer": {
            "values": ["SGD", "MGD", "NAG", "RMSPROP", "ADAM","NADAM"]
        },
                    
        "batch_size": {
            "values": [16, 32, 64]
        }
        
        
    }
}
sweep_id = wandb.sweep(sweep_config,project="cs6910_assignment1", entity="cs6910_assignment")


4431872/4422102 [==============================] - 0s 0us/step


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: rn6nojf0
Sweep URL: https://wandb.ai/cs6910_assignment/cs6910_assignment1/sweeps/rn6nojf0


In [ ]:

def train():    
    config_defaults = dict(
            max_epochs=10,
            num_hidden_layers=3,
            num_hidden_neurons=32,
            weight_decay=0,
            learning_rate=1e-3,
            optimizer="NADAM",
            batch_size=16,
            activation="SIGMOID",
            initializer="XAVIER",
            loss="MSE",
        )
    configdef = {
            'max_epochs':5,
            'num_hidden_layers':5,
            'num_hidden_neurons':128,
            'weight_decay':0.5,
            'learning_rate':'1e-4',
            'optimizer':"ADAM",
            'batch_size':64,
            'activation':"RELU",
            'initializer':"XAVIER",
            'loss':"MSE"
            }
        
    # wandb.init(config = config_defaults)
    wandb.init(project='cs6910_assignment1', entity='cs6910_assignment',config = configdef)

    wandb.run.name = "hl_" + str(wandb.config.num_hidden_layers) + "_hn_" + str(wandb.config.num_hidden_neurons) + "_opt_" + wandb.config.optimizer + "_act_" + wandb.config.activation + "_lr_" + str(wandb.config.learning_rate) + "_bs_"+str(wandb.config.batch_size) + "_init_" + wandb.config.initializer + "_ep_"+ str(wandb.config.max_epochs)+ "_l2_" + str(wandb.config.weight_decay) 
    CONFIG = wandb.config
    
    
  

    FFNN = FeedForwardNeuralNetwork(
        num_hidden_layers=CONFIG.num_hidden_layers,
        num_hidden_neurons=CONFIG.num_hidden_neurons,
        # num_hidden_layers=3,
        # num_hidden_neurons=128,
        X_train_raw=trainIn,
        Y_train_raw=trainOut,
        N_train = N_train,
        X_val_raw = validIn,
        Y_val_raw = validOut,
        N_val = N_validation,
        X_test_raw = testIn,
        Y_test_raw = testOut,
        N_test = N_test,
        optimizer = CONFIG.optimizer,
        batch_size = CONFIG.batch_size,
        weight_decay = CONFIG.weight_decay,
        learning_rate = CONFIG.learning_rate,
        max_epochs = CONFIG.max_epochs,
        activation = CONFIG.activation,
        initializer = CONFIG.initializer,
        loss = CONFIG.loss
        # optimizer = "NADAM",
        # batch_size = 16,
        # weight_decay = 0,
        # learning_rate = 1e-3,
        # max_epochs = 10,
        # activation = "SIGMOID",
        # initializer = 'XAVIER',
        # loss = 'MSE'
        )



    training_loss, trainingaccuracy, validationaccuracy, Y_pred_train = FFNN.optimizer(FFNN.max_epochs, FFNN.N_train, FFNN.batch_size, FFNN.learning_rate,FFNN.X_train,FFNN)
wandb.agent(sweep_id, train, count = 100)

wandb: Agent Starting Run: 95euihsd with config:
wandb: 	activation: TANH
wandb: 	batch_size: 32
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 5
wandb: 	num_hidden_neurons: 64
wandb: 	num_layers: 5
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0
wandb: Currently logged in as: cs6910_assignment (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch: 0, Loss: 3.354e-02, Training accuracy:0.84, Validation Accuracy: 0.84, Time: 66.37, Learning Rate: 1.000e-04
Epoch: 1, Loss: 2.288e-02, Training accuracy:0.85, Validation Accuracy: 0.85, Time: 56.43, Learning Rate: 1.000e-04
Epoch: 2, Loss: 2.106e-02, Training accuracy:0.86, Validation Accuracy: 0.86, Time: 56.54, Learning Rate: 1.000e-04
Epoch: 3, Loss: 2.004e-02, Training accuracy:0.87, Validation Accuracy: 0.87, Time: 57.21, Learning Rate: 1.000e-04
Epoch: 4, Loss: 1.931e-02, Training accuracy:0.87, Validation Accuracy: 0.87, Time: 56.65, Learning Rate: 1.000e-04


epoch,▁▃▅▆█
loss,█▃▂▁▁
trainingaccuracy,▁▄▆▇█
validationaccuracy,▁▅▇██
epoch,4
loss,0.01931
trainingaccuracy,0.87065
validationaccuracy,0.86867


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 2sqlhio6 with config:
wandb: 	activation: RELU
wandb: 	batch_size: 32
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 10
wandb: 	num_hidden_neurons: 32
wandb: 	num_layers: 3
wandb: 	optimizer: SGD
wandb: 	weight_decay: 0.5
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Run 2sqlhio6 errored: ValueError('The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()')
wandb: ERROR Run 2sqlhio6 errored: ValueError('The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()')
wandb: Agent Starting Run: ekl4zib5 with config:
wandb: 	activation: TANH
wandb: 	batch_size: 64
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 5
wandb: 	num_hidden_neurons: 64
wandb: 	num_layers: 3
wandb: 	optimizer: NADAM
wandb: 	weight_decay: 0.5
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


...

Epoch: 0, Loss: 1.130e-01, Training accuracy:0.26, Validation Accuracy: 0.27, Time: 106.95, Learning Rate: 1.000e-03
Epoch: 1, Loss: 7.363e-02, Training accuracy:0.40, Validation Accuracy: 0.41, Time: 107.54, Learning Rate: 1.000e-03
Epoch: 2, Loss: 6.628e-02, Training accuracy:0.44, Validation Accuracy: 0.44, Time: 107.84, Learning Rate: 1.000e-03
Epoch: 3, Loss: 6.256e-02, Training accuracy:0.47, Validation Accuracy: 0.48, Time: 114.22, Learning Rate: 1.000e-03
Epoch: 4, Loss: 6.041e-02, Training accuracy:0.48, Validation Accuracy: 0.48, Time: 109.15, Learning Rate: 1.000e-03


epoch,▁▃▅▆█
loss,█▃▂▁▁
trainingaccuracy,▁▅▇██
validationaccuracy,▁▆▇██
epoch,4
loss,0.06041
trainingaccuracy,0.48039
validationaccuracy,0.48183


wandb: Agent Starting Run: 4uzvxgid with config:
wandb: 	activation: RELU
wandb: 	batch_size: 16
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 5
wandb: 	num_hidden_neurons: 64
wandb: 	num_layers: 5
wandb: 	optimizer: SGD
wandb: 	weight_decay: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Run 4uzvxgid errored: ValueError('The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()')
wandb: ERROR Run 4uzvxgid errored: ValueError('The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()')
wandb: Agent Starting Run: 6ifxvmrn with config:
wandb: 	activation: SIGMOID
wandb: 	batch_size: 32
wandb: 	initializer: RANDOM
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 10
wandb: 	num_hidden_neurons: 128
wandb: 	num_layers: 5
wandb: 	optimizer: RMSPROP
wandb: 	weight_decay: 0
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Epoch: 0, Loss: 3.949e-02, Training accuracy:0.80, Validation Accuracy: 0.79, Time: 106.89, Learning Rate: 1.000e-03
Epoch: 1, Loss: 2.809e-02, Training accuracy:0.82, Validation Accuracy: 0.81, Time: 106.39, Learning Rate: 1.000e-03
Epoch: 2, Loss: 2.509e-02, Training accuracy:0.84, Validation Accuracy: 0.83, Time: 105.61, Learning Rate: 1.000e-03
Epoch: 3, Loss: 2.324e-02, Training accuracy:0.84, Validation Accuracy: 0.83, Time: 104.62, Learning Rate: 1.000e-03
Epoch: 4, Loss: 2.189e-02, Training accuracy:0.85, Validation Accuracy: 0.85, Time: 109.96, Learning Rate: 1.000e-03
Epoch: 5, Loss: 2.084e-02, Training accuracy:0.86, Validation Accuracy: 0.85, Time: 105.08, Learning Rate: 1.000e-03
Epoch: 6, Loss: 1.990e-02, Training accuracy:0.87, Validation Accuracy: 0.85, Time: 106.25, Learning Rate: 1.000e-03
